In [3]:
!pip install geopandas
import geopandas as gpd
import json

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [43]:
!git clone https://github.com/pnadelofficial/FallDHCourseMaterials

Cloning into 'FallDHCourseMaterials'...
remote: Enumerating objects: 429, done.
remote: Total 429 (delta 0), reused 0 (delta 0), pack-reused 429
Receiving objects: 100% (429/429), 79.28 MiB | 25.43 MiB/s, done.
Resolving deltas: 100% (200/200), done.
Checking out files: 100% (163/163), done.


In [3]:
import glob

In [4]:
for file in list(glob.glob('FallDHCourseMaterials/*.geojson')):
  print(file)

FallDHCourseMaterials/Pesia_satrapy_admin2_polygon.geojson
FallDHCourseMaterials/Romanum_Admin_FINAL.geojson


In [4]:
## JSON
with open('/content/Romanum_Admin_FINAL.geojson') as f:
  poly_features = json.load(f)

In [8]:
poly_features['features'][0]['geometry']['coordinates']

[[[[35.511749268137535, 23.60455512958421],
   [35.51174970085178, 23.604555301512452],
   [35.511749712578464, 23.604555445502307],
   [35.51063971260004, 23.60824944614921],
   [35.50866771301869, 23.60852844630967],
   [35.508728991922155, 23.603807383738182],
   [35.511749268137535, 23.60455512958421]]],
 [[[35.50869371222584, 23.803388445811894],
   [35.51274871255905, 23.804861446111843],
   [35.51066571297986, 23.80997244583299],
   [35.51283271306452, 23.814945446113],
   [35.51189071292435, 23.816805445961506],
   [35.509861712329915, 23.815944445653916],
   [35.50535971232233, 23.80783244616542],
   [35.503582712071626, 23.798944445762697],
   [35.50869371222584, 23.803388445811894]]],
 [[[35.60713971303819, 23.825056445652475],
   [35.606304712927624, 23.829666445639],
   [35.60311171219734, 23.831361445647637],
   [35.60083371211485, 23.822944445718093],
   [35.60163871305001, 23.81719444641921],
   [35.59877771202803, 23.813612445616183],
   [35.601722712382816, 23.8126104

In [6]:
pd.Series(poly_features['features'][1]['geometry']['coordinates']).apply(lambda x: np.mean(np.array(x), axis = 0)).mean()

array([34.95936254, 30.28591759])

In [5]:
import pandas as pd
import numpy as np

avg_points = []
for feature in poly_features['features']:
  if feature['geometry']['type'] == 'MultiPolygon':
    avg_points.append(pd.Series(feature['geometry']['coordinates']).apply(lambda x: np.mean(np.array(x[0]), axis = 0)).mean())
  else:
    avg_points.append(pd.Series(feature['geometry']['coordinates']).apply(lambda x: np.mean(np.array(x), axis = 0)).mean())

avg_points = np.array(avg_points)
np.mean(avg_points, axis=0)

array([20.27161338, 40.49041372])

In [86]:
for feature in poly_features['features']:
  print(feature['geometry']['type'])

MultiPolygon
MultiPolygon
MultiPolygon
MultiPolygon
Polygon
Polygon
MultiPolygon
MultiPolygon
MultiPolygon
MultiPolygon
Polygon
Polygon
Polygon
MultiPolygon
Polygon
MultiPolygon
Polygon
Polygon
Polygon
Polygon
Polygon
Polygon
MultiPolygon
MultiPolygon
MultiPolygon
Polygon
MultiPolygon
MultiPolygon
MultiPolygon
Polygon
Polygon
Polygon
MultiPolygon
MultiPolygon
MultiPolygon
MultiPolygon
Polygon
Polygon


In [ ]:
import plotly.graph_objects as go

fig = go.Figure(go.Scattermapbox(
    mode = "markers",
    lon = [-73.605], lat = [45.51],
    marker = {'size': 20, 'color': ["cyan"]}))

fig.update_layout(
    mapbox = {
        'style': "stamen-terrain",
        'center': { 'lon': 30, 'lat': 42},
        'zoom': 3, 'layers': [{
            'source':
                poly_features,
            'type': "fill", 'below': "traces", 'color': "royalblue"}]},
    margin = {'l':0, 'r':0, 'b':0, 't':0})

fig.show()

In [15]:
%%writefile util.py
import plotly.graph_objects as go
import json
import pandas as pd
import numpy as np

def openJSON(file_path):
  with open(file_path) as f:
    poly = json.load(f)
  return poly

def plotlyPlot(poly):
  avg_points = []
  for feature in poly['features']:
    if feature['geometry']['type'] == 'MultiPolygon':
      avg_points.append(pd.Series(feature['geometry']['coordinates']).apply(lambda x: np.mean(np.array(x[0]), axis = 0)).mean())
    else:
      avg_points.append(pd.Series(feature['geometry']['coordinates']).apply(lambda x: np.mean(np.array(x), axis = 0)).mean())
  avg_points = np.array(avg_points)
  avg_point = np.mean(avg_points, axis=0)

  fig = go.Figure(go.Scattermapbox(
    mode = "markers",
    marker = {'size': 20, 'color': ["cyan"]}))

  fig.update_layout(
      mapbox = {
          'style': "stamen-terrain",
          'center': { 'lon': float(avg_point[0]), 'lat': float(avg_point[1])},
          'zoom': 3, 'layers': [{
              'source':
                  poly,
              'type': "fill", 'below': "traces", 'color': "royalblue"}]},
      margin = {'l':0, 'r':0, 'b':0, 't':0})
  
  return fig

Overwriting util.py


In [16]:
%%writefile dashboard.py
import geopandas as gpd
import streamlit as st
import glob
from util import *

geojson_list = []
for file in list(glob.glob('FallDHCourseMaterials/*.geojson')):
  geojson_list.append(file)

geo_select = st.radio(
     "Select map",
     geojson_list)

if geo_select == geojson_list[0]:
  poly = openJSON(geojson_list[0])
  fig = plotlyPlot(poly)
  st.plotly_chart(fig, use_container_width=True)
else:
  poly = openJSON(geojson_list[1])
  fig = plotlyPlot(poly)
  st.plotly_chart(fig, use_container_width=True)

Overwriting dashboard.py


In [17]:
!streamlit run dashboard.py & npx localtunnel --port 8501

2022-05-31 19:14:17.259 INFO    numexpr.utils: NumExpr defaulting to 2 threads.
npx: installed 22 in 1.783s
your url is: https://warm-readers-tease-34-90-128-26.loca.lt

  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.2:8501
  External URL: http://34.90.128.26:8501

  Stopping...
^C
